# Reading from an Icechunk Store

This notebook demonstrates how to open an existing Icechunk store (created by `virtualizarr_recipes.ipynb`) and load a subset of the CCMP wind data.

**Prerequisites:** Run Sections 1-2.2 of `virtualizarr_recipes.ipynb` first to create the `ccmp_icechunk_1year` store.

## Setup

In [1]:
import warnings

import earthaccess
import icechunk
import matplotlib.pyplot as plt
import xarray as xr
import zarr
from zarr.storage import LoggingStore

warnings.filterwarnings(
    "ignore",
    message="Numcodecs codecs are not in the Zarr version 3 specification.*",
    category=UserWarning,
)

xr.set_options(
    display_expand_attrs=False,
    display_expand_coords=True,
    display_expand_data=True,
)

## Increase zarr's max concurrency

In [2]:
zarr.config.set({"async.concurrency": 1})

## Open the Icechunk store

The Icechunk store contains virtual references to the original CCMP data files on S3. To read the actual data, Icechunk needs S3 credentials for the PO.DAAC bucket. We reuse the same repository configuration that was set up during the write step.

In [3]:
earthaccess.login()

# The bucket URL must match what was used when creating the store.
# If you don't know it, you can re-derive it from earthaccess:
granule_info = earthaccess.search_data(
    short_name="CCMP_WINDS_10M6HR_L4_V3.1",
    count=1,
)
first_link = granule_info[0].data_links(access="direct")[0]
bucket = "/".join(first_link.split("/")[:3])
print(f"Bucket: {bucket}")

Bucket: s3://podaac-ops-cumulus-protected


In [4]:
# Configure the repository to know how to read virtual chunks from S3:
config = icechunk.RepositoryConfig.default()
config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(
        bucket + "/",
        icechunk.s3_store(region="us-west-2"),
    )
)

# Get temporary S3 credentials for reading the actual data:
s3_creds = earthaccess.get_s3_credentials(daac="PODAAC")
credentials = icechunk.containers_credentials(
    {
        bucket + "/": icechunk.s3_credentials(
            access_key_id=s3_creds["accessKeyId"],
            secret_access_key=s3_creds["secretAccessKey"],
            session_token=s3_creds["sessionToken"],
        )
    }
)

# Open the existing repository:
storage = icechunk.local_filesystem_storage("./ccmp_icechunk_1year")
repo = icechunk.Repository.open(
    storage=storage,
    config=config,
    authorize_virtual_chunk_access=credentials,
)

  2026-02-11T00:40:23.347166Z  WARN icechunk::storage::object_store: The LocalFileSystem storage is not safe for concurrent commits. If more than one thread/process will attempt to commit at the same time, prefer using object stores.
    at icechunk/src/storage/object_store.rs:81



## Lazy-load the full dataset

Opening with `xr.open_zarr` is lazy -- no data is read from S3 yet. This should take only a few seconds even for the full year.

In [5]:
session = repo.readonly_session(branch="main")

store = LoggingStore(session.store)

In [6]:
%%time
ds = xr.open_zarr(
    store=store,
    zarr_format=3,
    consolidated=False,
)
ds

2026-02-11 00:40:23,970 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO -  Calling IcechunkStore.read_only
2026-02-11 00:40:23,971 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO - Finished IcechunkStore.read_only [0.00 s]
2026-02-11 00:40:23,973 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO -  Calling IcechunkStore._ensure_open
2026-02-11 00:40:23,973 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO - Finished IcechunkStore._ensure_open [0.00 s]
2026-02-11 00:40:23,976 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO -  Calling IcechunkStore.get(zarr.json)
2026-02-11 00:40:23,977 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO - Finished IcechunkStore.get(zarr.json) [0.00 s]
2026-02-11 00:40:24,002 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO -  Calling IcechunkStore.r

<xarray.Dataset> Size: 24GB
Dimensions:    (time: 1464, latitude: 720, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 6kB 0.125 0.375 0.625 ... 359.4 359.6 359.9
  * time       (time) datetime64[ns] 12kB 1993-01-02 ... 1994-01-05T18:00:00
Data variables:
    nobs       (time, latitude, longitude) float32 6GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    uwnd       (time, latitude, longitude) float32 6GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vwnd       (time, latitude, longitude) float32 6GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ws         (time, latitude, longitude) float32 6GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (54)

## Select and load a subset

With lazy loading, we can select a spatial and temporal subset before any data is fetched from S3. Only the chunks that overlap the selection are read.

### Single time step, global map

In [7]:
%%time
# Select the first time step and load wind speed into memory:
snapshot = ds["ws"].isel(time=0)

CPU times: user 1.03 ms, sys: 150 μs, total: 1.18 ms
Wall time: 1.19 ms


In [ ]:
snapshot.isel(latitude=0, longitude=0).values

2026-02-11 00:40:25,138 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO -  Calling IcechunkStore.get(ws/c/0/0/0)
2026-02-11 00:40:25,565 - LoggingStore(<icechunk.store.IcechunkStore object at 0x7fe758493a10>) - INFO - Finished IcechunkStore.get(ws/c/0/0/0) [0.43 s]


In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
snapshot.plot(ax=ax, cmap="viridis", vmin=0, vmax=20)
ax.set_title(f"CCMP Wind Speed — {snapshot.time.values}")
plt.tight_layout()
plt.show()

### Spatial subset over time

Select a region (e.g. the tropical Pacific) across all time steps. Only the chunks covering this region are fetched.

In [ ]:
%%time
# Tropical Pacific region: 10°S–10°N, 160°E–280°E (= 160°E–80°W)
region = ds["ws"].sel(latitude=slice(-10, 10), longitude=slice(160, 280))
print(f"Subset shape: {dict(region.sizes)}")
print(f"Full dataset shape: {dict(ds['wspd'].sizes)}")

region_loaded = region.load()

In [ ]:
# Hovmöller diagram: longitude-averaged wind speed over time
fig, ax = plt.subplots(figsize=(12, 4))
region_loaded.mean(dim="longitude").plot(ax=ax, cmap="viridis", vmin=0, vmax=15)
ax.set_title("CCMP Wind Speed — Tropical Pacific (10°S–10°N), longitude-averaged")
ax.set_ylabel("Latitude")
plt.tight_layout()
plt.show()

### Time range subset

Select a month of data and compute a temporal mean.

In [ ]:
%%time
# Select January and compute the monthly mean wind speed:
jan = ds["ws"].sel(time=ds.time.dt.month == 1)
print(f"January time steps: {jan.sizes['time']}")

jan_mean = jan.mean(dim="time").load()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
jan_mean.plot(ax=ax, cmap="viridis", vmin=0, vmax=15)
ax.set_title("CCMP Mean Wind Speed — January")
plt.tight_layout()
plt.show()

### Point time series

Extract a time series at a single location (e.g. a buoy site).

In [ ]:
%%time
# Time series at a point in the North Atlantic (~30°N, 330°E = 30°W):
point = ds["ws"].sel(latitude=30, longitude=330, method="nearest").load()
point

In [ ]:
fig, ax = plt.subplots(figsize=(12, 3))
point.plot(ax=ax, linewidth=0.5)
ax.set_title(
    f"CCMP Wind Speed — {float(point.latitude):.1f}°N, {360 - float(point.longitude):.1f}°W"
)
ax.set_ylabel("Wind speed (m/s)")
plt.tight_layout()
plt.show()

## Summary

This notebook showed how to:

1. **Open** an existing Icechunk store with the required S3 credentials for virtual chunk access.
2. **Lazy-load** the full year of CCMP data (no data fetched until `.load()` or `.compute()`).
3. **Select subsets** by time, space, or both — only the relevant chunks are read from S3.
4. **Compute** reductions (temporal means) and **plot** the results.

The key takeaway: Icechunk + xarray gives you the same subsetting and computation experience as if the data were a single local Zarr store, but the actual bytes live in the original NetCDF4/HDF5 files on S3.